### FUNCIONES TEST

- ``row_counting_deviation(df, previous_df, deviation_threshold = 0.1)``: Test que chequea si un dataframe tiene una desviacion en el numero de filas mayor a lo indicado en deviation_threshold con respecto a otro dataframe previo
- ``column_counting_deviation(df, previous_df, deviation_threshold = 0.0)``: Test que chequea si un dataframe tiene una desviacion en el numero de columnas mayor a lo indicado en deviation_threshold con respecto a otro dataframe previo
- ``column_names_checking(df, previous_df)``: Test que chequea si los nombres de las columnas de un dataframe son iguales a las del dataframe previo
- ``duplicated_key(df, columns_list)``: Test que chequea si hay duplicados en la clave unica de un dataframe
- ``nulls_check(df, columns_list)``: Test que chequea si hay valores nulos en una serie de columnas
- ``sum_column_values_deviation(df, previous_df, columns_list, deviation_threshold = 0.1)``: Test que chequea si la suma del total de los valores de una serie de columnas se desvia en mas de deviation_threshold con respecto a un dataframe previo
- ``sum_column_aggregated_values_deviation(df, previous_df, columns_list, aggregation_keys, deviation_threshold = 0.1)``: Test que chequea si la suma agregada por variables dadas de los valores de una serie de columnas se desvia en mas de deviation_threshold con respecto a un dataframe previo
- ``categories_counting_deviation(df, previous_df, columns_list, deviation_threshold = 0.1)``: Test que imprime la distribucion de las variables definidas en proporcion de sus categorias. Si se indica un previous_df ademas comprobara diferencias en las categorias con la distribucion de ese df anterior. Si se supera el umbral dado generara un error
- ``range_control(df, columns_list, range_list)``: Test que chequea si los valores de una serie de columnas estan entre un rango dado, en caso de variables categoricas, chequea que todos los valores de la lista dada estan en la variable del dataframe, y no hay ninguna categoria extra en el
- ``run_all_tests(df, dict_argumentos)``: Ejecuta todos los tests que indiquemos en test_executions de dict_argumentos de manera automatica. Debemos darle los valores de entrada de los tests en forma de diccionarios con el nombre del test como clave en string y los valores como valor. 
- ``total_columns_sum_check(df, dict_sum_of_columns):``: Test que comprueba si la columna que deberia equivaler a la suma de diferentes columnas esta bien calculado. 
- ``variables_type(df, columns_list, types_list)``: Test que chequea si el formato de una lista de variables coincide con el que nosotros especifiquemos 
- ``variable_distribution(df, previous_df, columns_list, deviation_threshold = 0.1)``: Test que chequea si hay desviaciones en los p0 p25 p40 p50 p60 p75 p100 y la media de distribucion de un df con uno previo. Si no se da un datarame previo se pintara la distribucion del actual
- ``criteria_pointed_check(df, specifications)``: Test que chequea si la puntuacion de los agentes esta bien calculada con respecto a las condiciones que le corresponderian a cada uno de esos agentes.
- ``round_numeric_vars(df, exclude_cols = [], scale = 2)``: Función que redondea los valores numéricos decimales de un DataFrame al tipo DecimalType(18, scale). Las columnas incluidas en el parámetro de entrada 'exclude_cols' no son transformadas independientemente de su tipo inicial.
- ``check_equal_dfs(df1, df2, exclude_conv_cols=[], scale_num_conv=2)``: Test que chequea si dos DataFrames son iguales. Las columnas numéricas decimales de ambos DataFrames son transformadas de acuerdo con la función ``round_numeric_vars``.
- ``saving_after_testing(df, environment, output_table_name, unique_key='cod_mediador', carga = 'overwrite')``: Funcion que define la ruta de guardado de un proceso dependiendo del tipo de testeo que se este ejecutando.
- ``saving_after_testing_dlake(df, environment, output_table_name, table_format, partitioned_by=None, replace_where=None, container=None, carga ='overwrite')``: Funcion que define la ruta de guardado de un proceso dependiendo del tipo de testeo que se este ejecutando (entorno de ejecución Data Lake).

In [0]:
import datavault as dv
import analytics.general_utils as gen_utils
import analytics.ml_pipeline as ml_pipe
from pyspark.sql import functions as F
from pyspark.sql import types as T
from pyspark.sql import Row
import os

from datetime import datetime
import pandas as pd

In [0]:
def row_counting_deviation(df, previous_df, deviation_threshold = 0.1):
  
  """
  Test que chequea si un dataframe tiene una desviacion en el numero de filas mayor a lo indicado en deviation_threshold con respecto a otro dataframe previo
  
  Parameters:
  ------------
  df : Spark DataFrame
    DataFrame actual
  previous_df : Spark DataFrame
    DataFrame previo contra el que queremos comprobar
  deviation_threshold : Float
    Desviacion en decimal
  
  Returns:
  --------
  Mensaje de error y la ejecucion se para si la desviacion es mayor. Mensaje de OK si pasa el test 
  """

  current_df_count = df.count()
  previous_df_count = previous_df.count()
  deviation = abs((current_df_count - previous_df_count) / previous_df_count)
  if deviation >= deviation_threshold:
    print('{0: .2f}'.format(deviation * 100) + '%')
    raise SystemExit('Hay una desviacion igual o mayor al {}% en el numero de filas de la nueva tabla con respecto a la version previa'.format(deviation_threshold * 100))
  
  else:
    return 'Desviacion de numero de filas dentro de limites OK, desviacion del {0:.0f}%'.format(deviation * 100)
  
#---------------------------------------------------------------------------------

def column_counting_deviation(df, previous_df, deviation_threshold = 0.0):
  
  """
  Test que chequea si un dataframe tiene una desviacion en el numero de columnas mayor a lo indicado en deviation_threshold con respecto a otro dataframe previo
  
  Parameters:
  ------------
  df : Spark DataFrame
    DataFrame actual
  previous_df : Spark DataFrame
    DataFrame previo contra el que queremos comprobar
  deviation_threshold : Float
    Desviacion en decimal
  
  Returns:
  --------
  Mensaje de error y la ejecucion se para si la desviacion es mayor. Mensaje de OK si pasa el test
  """
  
  current_df_count = 0
  previous_df_count = 0
  
  for column in df.columns:
    current_df_count += 1
    
  for column in previous_df.columns:
    previous_df_count += 1
    
  deviation = abs((current_df_count - previous_df_count) / previous_df_count)
  
  if deviation > 0:
    
    current_df_cols = [col.upper() for col in df.columns]
    previous_df_cols = [col.upper() for col in previous_df.columns]

    lost_cols = [col for col in previous_df_cols if col not in current_df_cols]
    new_cols = [col for col in current_df_cols if col not in previous_df_cols]
        
    if deviation > deviation_threshold:
      raise SystemExit('Hay una desviacion igual o mayor al {0:.0f}% en el numero de columnas de la nueva tabla con respecto a la version previa. Las columnas {1} son nuevas y las columnas {2} se han eliminado.'.format(deviation_threshold * 100, new_cols, lost_cols))

    else:
      return 'Desviacion en numero de columnas dentro de limites OK, desviacion del {0:.0f}%. Las columnas {1} son nuevas y las columnas {2} se han eliminado.'.format(deviation * 100, new_cols, lost_cols)
    
  else:
    return 'Desviacion en numero de columnas OK, desviacion del {0:.0f}%'.format(deviation * 100)

#---------------------------------------------------------------------------------

def column_names_checking(df, previous_df):
  
  """
  Test que chequea si los nombres de las columnas de un dataframe son iguales a las del dataframe previo
  
  Parameters:
  ------------
  df : Spark DataFrame
    DataFrame actual
  previous_df : Spark DataFrame
    DataFrame previo contra el que queremos comprobar
  
  Returns:
  --------
  Mensaje de error y la ejecucion se para si hay columnas con nombres diferentes. Mensaje de OK si pasa el test
  """
  
  current_df_cols = []
  previous_df_cols = []
  new_cols = []
  lost_cols = []
  
  current_df_cols = [col.upper() for col in df.columns]
  previous_df_cols = [col.upper() for col in previous_df.columns]
    
  lost_cols = [col for col in previous_df_cols if col not in current_df_cols]
  new_cols = [col for col in current_df_cols if col not in previous_df_cols]
      
  if len(lost_cols) > 0:
    raise SystemExit('Las columnas {} no estan incluidas en la nueva tabla'.format(lost_cols))
      
  elif len(new_cols) > 0:
    raise SystemExit('La columnas {} no estaban incluidas en la tabla anterior'.format(new_cols))
  
  else:
    return 'Nombres de columnas sin modificaciones OK'

#---------------------------------------------------------------------------------

def duplicated_key(df, columns_list):
  
  """
  Test que chequea si hay duplicados en la clave unica de un dataframe
  
  Parameters:
  ------------
  df : Spark DataFrame
    DataFrame actual
  columns_list : String / list of strings
    Lista con las columnas que crean la clave unica
  
  Returns:
  --------
  Mensaje de error y la ejecucion se para si hay duplicados. Mensaje de OK si pasa el test
  """
  
  if type(columns_list) is str:
    columns_list = [columns_list]
  
  grouped_data = df.groupBy(columns_list).count()
  duplicated_rows = grouped_data.where(F.col('count') > 1).count()
  
  if duplicated_rows > 0:
    grouped_data.orderBy('count', ascending = False).show()
    raise SystemExit('Hay valores de la key proporcionada en row_list duplicados. Se muestran ejemplos en la tabla.')

  else:
    return 'No duplicados en la clave unica OK'
  
#---------------------------------------------------------------------------------

def nulls_check(df, columns_list = None):
  
  """
  Test que chequea si hay valores nulos en una serie de columnas
  
  Parameters:
  ------------
  df : Spark DataFrame
    DataFrame actual
  columns_list : String / list of strings
    Lista con las columnas que se quieren testear
  
  Returns:
  --------
  Mensaje de error y la ejecucion se para si hay nulos. Mensaje de OK si pasa el test
  """
  
  if columns_list == None:
    columns_list = [col for col in df.columns]
  
  elif type(columns_list) is str:
    columns_list = [columns_list]
  
  columns_with_nulls = []
  
  for column in columns_list:
    if df.select(column).where(F.col(column).isNull()).count() > 0:
      columns_with_nulls.append(column)
  
    if len(columns_with_nulls) > 0:
      raise SystemExit('Hay valores nulos en la columna {}'.format(column))
  
  
  return 'No nulos OK'

#---------------------------------------------------------------------------------

def sum_column_values_deviation(df, previous_df, columns_list, deviation_threshold = 0.1, nivel_mediador = False):
  
  """
  Test que chequea si la suma del total de los valores de una serie de columnas se desvia en mas de deviation_threshold con respecto a un dataframe previo
  
  Parameters:
  ------------
  df : Spark DataFrame
    DataFrame actual
  previous_df : Spark DataFrame
    DataFrame anterior contra el que queremos comparar
  columns_list : String / list of strings
    Lista con las columnas que se quieren testear
  deviation_threshold : Float
    Desviacion en decimal
  nivel_mediador : boolean
    Si esta en True, generara ademas un test a nivel mediador de las columnas incluidas. Por defecto False
  
  Returns:
  --------
  Mensaje de error y la ejecucion se para si hay alguna columna con una desviacion mayor a deviation_threshold, muestra la columna que se ha desviado. Mensaje de OK si pasa el test
  """
  
  previous_df = gen_utils.convert_numeric_variables_to_float(previous_df)
  df = gen_utils.convert_numeric_variables_to_float(df)
  
  if type(columns_list) is str:
    columns_list = [columns_list]
    
  for column in columns_list:
    sum_current_column_values = df.select(column).groupBy().agg(F.sum(column)).collect()[0][0]
    sum_previous_column_values = previous_df.select(column).groupBy().agg(F.sum(column)).collect()[0][0]
    deviation = abs((sum_current_column_values - sum_previous_column_values) / sum_previous_column_values)
    if deviation > deviation_threshold:
      raise SystemExit('Hay una desviacion de mas del {0:.0f}% en la suma de valores de la columna {1}'.format(deviation_threshold * 100, column))
    
    elif deviation > 0:
      print('Desviacion del {0:.0f}% en la suma de valores de la columna {1} inferior al limite. Valor anterior {2:.2f}, valor actual {3:.2f}.'.format(deviation * 100, column, sum_previous_column_values, sum_current_column_values))
    
    if nivel_mediador == True:
      sum_current_column_values = df.groupBy('cod_mediador').agg(F.sum(column)).withColumnRenamed('sum(' + column + ')',  column + '_current')
      sum_previous_column_values = previous_df.groupBy('cod_mediador').agg(F.sum(column)).withColumnRenamed('sum(' + column + ')', column + '_prev')
      joined_values = sum_current_column_values.join(sum_previous_column_values, how = 'full', on = 'cod_mediador').na.fill(0)\
                                               .withColumn('diff', F.udf(lambda x, y: 0 if x == 0 else abs((float(x)- float(y)) / float(x)), T.FloatType())(column + '_current', column + '_prev'))
      
      if joined_values.where(F.col('diff') > deviation_threshold).count() > 0 :
        joined_values.where(F.col('diff') > deviation_threshold).show()
        raise SystemExit('Hay una desviacion de mas del {0:.0f}% en la columna {1} en {2} mediadores. Se muestran casos.'.format(deviation_threshold * 100, column, joined_values.where(F.col('diff') > deviation_threshold).count()))

      elif deviation > 0:
        joined_values.where((F.col('diff') > 0) & (F.col('diff') <= deviation_threshold)).show()
        print('Hay una desviacion menor al {0:.0f}% en la columna {1} en {2} mediadores. Se muestran casos.'.format(deviation_threshold * 100, column, joined_values.where((F.col('diff') > 0) & (F.col('diff') <= deviation_threshold)).count()))
      
      else:
        print('Desviacion en columna {0} para todos los mediadores OK'.format(column))

  return 'Desviacion de suma de valores de columnas OK'

#---------------------------------------------------------------------------------

def sum_column_aggregated_values_deviation(df, previous_df, columns_list, aggregation_keys, deviation_threshold = 0.1):
  
  """
  Test que chequea si la suma agregada por variables dadas de los valores de una serie de columnas se desvia en mas de deviation_threshold con respecto a un dataframe previo
  
  Parameters:
  ------------
  df : Spark DataFrame
    DataFrame actual
  previous_df : Spark DataFrame
    DataFrame anterior contra el que queremos comparar
  columns_list : String / list of strings
    Lista con las columnas que se quieren testear
  aggregation_keys : String / list of strings
    Las variables por las que queremos que agrupe la suma
  deviation_threshold : Float
    Desviacion en decimal
  
  Returns:
  --------
  Mensaje de error y la ejecucion se para si hay alguna columna con una desviacion mayor a deviation_threshold, muestra una tabla con los valores que mas se devian. Mensaje de OK si pasa el test
  """
  
  previous_df = gen_utils.convert_numeric_variables_to_float(previous_df)
  df = gen_utils.convert_numeric_variables_to_float(df)
  
  if type(columns_list) is str:
    columns_list = [columns_list]
  
  for column in columns_list:
    sum_current_column_values = df.groupBy(aggregation_keys).agg(F.sum(column).alias(column + '_current'))
    sum_previous_column_values = previous_df.groupBy(aggregation_keys).agg(F.sum(column).alias(column + '_prev'))
    
    joined_values = sum_previous_column_values.join(sum_current_column_values, on = aggregation_keys, how = 'full').na.fill(0)\
                                             .withColumn('diff', F.udf(lambda x, y: 0 if x == 0 else abs((float(y) - float(x)) / float(x)), T.FloatType())(column + '_prev', column + '_current'))
    
    deviated_rows = joined_values.where(F.col('diff') > deviation_threshold)
    
    if deviated_rows.count() > 0:
      deviated_rows.orderBy('diff', ascending = False).show()
      raise SystemExit('Hay una desviacion de mas del {0:.0f}% para la columna {1} en las agregaciones mostradas en la tabla'.format(deviation_threshold * 100, column))
    elif joined_values.where(F.col('diff') > 0).count() > 0:
      joined_values.where(F.col('diff') > 0).orderBy('diff', ascending = False).show()
      print('Hay desviaciones en suma de valores agregados que son inferiores al limite {0:.0f}% en la columna {1}. En la tabla se muestran algunos casos. Desviacion en suma de valores agregados de columnas dentro de limites OK'.format(deviation_threshold * 100, column))
  
  return 'Desviacion en suma de valores agregados de OK.'

#---------------------------------------------------------------------------------

def categories_counting_deviation(df, previous_df, columns_list, deviation_threshold = 0.1):
  
  """
  Test que imprime la distribucion de las variables definidas en proporcion de sus categorias. Si se indica un previous_df ademas comprobara diferencias en las categorias con la distribucion de ese df anterior. Si se supera el umbral dado generara un error
  
  Parameters:
  ------------
  df : Spark DataFrame
    DataFrame actual
  previous_df : Spark DataFrame
    DataFrame anterior contra el que queremos comparar
  columns_list : String / list of strings
    Lista con las columnas que se quieren testear
  deviation_threshold : Float
    Desviacion en decimal
  
  Returns:
  --------
  Distribucion de las categorioas del df actal. Si hay previous_df mensaje de error y la ejecucion se para si hay alguna columna con una desviacion mayor a deviation_threshold, muestra una tabla con los valores que mas se devian. Mensaje de OK si pasa el test
  """
  
  if type(columns_list) is str:
      columns_list = [columns_list]
  
  if previous_df == None:
    df = gen_utils.convert_numeric_variables_to_float(df)

    for column in columns_list:
      current_category_count = df.groupBy(column).count().withColumnRenamed('count', column + '_current_count')
      current_total_count = df.groupBy().count().collect()[0][0]
      current_category_count = current_category_count.withColumn('total_count', F.lit(current_total_count))\
                                                     .withColumn('category_per_current', F.udf(lambda x, y: (float(x) / float(y) * 100) , T.FloatType())(column + '_current_count', 'total_count'))\
                                                     .drop('total_count')  

      print('Distribucion de la variable {}:'.format(column))
      current_category_count.show()
  
  elif previous_df != None:
    previous_df = gen_utils.convert_numeric_variables_to_float(previous_df)

    for column in columns_list:
      current_category_count = df.groupBy(column).count().withColumnRenamed('count', column + '_current_count')
      current_total_count = df.groupBy().count().collect()[0][0]
      current_category_count = current_category_count.withColumn('total_count', F.lit(current_total_count))\
                                                     .withColumn('category_per_current', F.udf(lambda x, y: (float(x) / float(y) * 100) , T.FloatType())(column + '_current_count', 'total_count'))\
                                                     .drop('total_count')
      
      print('Distribucion de la variable {}:'.format(column))
      current_category_count.show()
      
      previous_category_count = previous_df.groupBy(column).count().withColumnRenamed('count', column + '_prev_count')
      previous_total_count = previous_df.groupBy().count().collect()[0][0]
      previous_category_count = previous_category_count.withColumn('total_count', F.lit(previous_total_count))\
                                                       .withColumn('category_per_prev', F.udf(lambda x, y: (float(x) / float(y) * 100) , T.FloatType())(column + '_prev_count','total_count'))\
                                                       .drop('total_count')

      joined_values = previous_category_count.join(current_category_count, how = 'full', on = column).na.fill(0)\
                                            .withColumn('diff', F.udf(lambda x, y: 0 if x == 0 else abs((float(y) - float(x)) / float(x)), T.FloatType())('category_per_prev', 'category_per_current'))

      deviated_rows = joined_values.where(F.col('diff') > deviation_threshold)

      if deviated_rows.count() > 0:
        deviated_rows.orderBy('diff', ascending = False).show()
        raise SystemExit('Hay una desviacion de mas del {0:.0f}% para la columna {1} en las categorias mostradas en la tabla'.format(deviation_threshold * 100, column))
      elif joined_values.where(F.col('diff') > 0).count() > 0: 
        print('Diferencias en la distribucion de la variable {}:'.format(column))
        joined_values.where(F.col('diff') > 0).orderBy('diff', ascending = False).show()
        print('Hay desviaciones en distribucion de la columna {0} que son inferiores al limite {1:.0f}%. En la tabla se muestran algunos casos.'.format(column, deviation_threshold * 100))

  return 'Distribuciones por categoria dentro de limites OK'

#---------------------------------------------------------------------------------

def range_control(df, columns_list, range_list):
  
  """
  Test que chequea si los valores de una serie de columnas estan entre un rango dado, en caso de variables categoricas, chequea que todos los valores de la lista dada estan en la variable del dataframe, y no hay ninguna categoria extra en el
  
  Parameters:
  ------------
  df : Spark DataFrame
    DataFrame actual
  columns_list : String / list of strings
    Lista con las columnas que se quieren testear
  range_list : list of floats / list of strings / list of lists
    Lista con el o los rangos de las columnas que queremos testear. En caso de variables numericas se indicaran los valores maximos y minimos, en caso de categoricas se indicaran todas las categorias que la variable debe tener

  Returns:
  --------
  Mensaje de error y la ejecucion se para si hay alguna columna que salga del rango o que tenga categorias diferentes a las indicadas, muestra una tabla con los valores que mas se devian. Mensaje de OK si pasa el test
  """
    
  if type(columns_list) is str:
    columns_list = [columns_list]
  
  if type(range_list[0]) is not list:
    range_list = [range_list]
    
  counter = 0
  
  for column in columns_list:
    if type(range_list[counter][0]) is str:
      elements_in_df = df.select(column).dropDuplicates().toPandas()
      for element in range_list[counter]:
        if element in set(elements_in_df[column]) ==  False:
          raise SystemExit('El valor {} no esta en contenido en la columna {}'.format(element, column))

      for element in set(elements_in_df[column]):
        if element in range_element ==  False:
          raise SystemExit('El valor {} esta en contenido en la columna {} y no es parte del rango de valores definidos'.format(element, column))

    elif type(range_list[counter][0]) is not str:
      max_bound = max(range_list[counter])
      min_bound = min(range_list[counter])
      outbounded_values = df.where((F.col(column) < min_bound) | (F.col(column) > max_bound))
      if outbounded_values.count() > 0:
        raise SystemExit('Hay valores fuera de rango en la columna {}'.format(column))

    counter += 1

  return 'Control de rangos y valores OK'

#---------------------------------------------------------------------------------

def run_all_tests(df, dict_argumentos):
  
  """
  Ejecuta todos los tests que indiquemos en test_executions de dict_argumentos de manera automatica. Debemos darle los valores de entrada de los tests en forma de diccionarios con el nombre del test como clave en string y los valores como valor. 
  
  Ejemplo:
  
  dict_argumentos = {'tests': tests, 
                   'previous_df': df_comparacion,
                   'dict_args_columns_lists': argumentos_columnas,
                   'dict_ranges': argumentos_rangos,
                   'dict_thresholds': argumentos_desviaciones,
                   'dict_total_sum': argumentos_columna_suma_total,
                   'dict_specifications': dict_especificaciones}
  
  Parameters:
  ------------
  df : Spark DataFrame
    DataFrame actual
  dict_argumentos : dict of dicts
    Lista con los diccionarios que queremos meter en la funcion

  Returns:
  --------
  Devuelve la salida de cada uno de los tests que se ejecutan. Si alguno falla, la ejecucion se para
  """
  
  if 'tests' in dict_argumentos.keys():
    test_executions = dict_argumentos['tests']
  else:
    raise SystemExit('No se ha especificado ningun test')
  
  if 'previous_df' in dict_argumentos.keys():
    previous_df = dict_argumentos['previous_df']
  else:
    previous_df = None
  if 'dict_args_columns_lists' in dict_argumentos.keys():
    dict_args_columns_lists = dict_argumentos['dict_args_columns_lists']
  if 'dict_args_aggs' in dict_argumentos.keys():
    dict_args_aggs = dict_argumentos['dict_args_aggs']
  if 'dict_ranges' in dict_argumentos.keys():
    dict_ranges = dict_argumentos['dict_ranges']
  if 'dict_thresholds' in dict_argumentos.keys():
    dict_thresholds = dict_argumentos['dict_thresholds']
  else:
    dict_thresholds = {}
  if 'dict_types' in dict_argumentos.keys():
    dict_types = dict_argumentos['dict_types']
  if 'dict_total_sum' in dict_argumentos.keys():
    dict_total_sum = dict_argumentos['dict_total_sum']
  if 'dict_specifications' in dict_argumentos.keys():
    dict_specifications = dict_argumentos['dict_specifications']
  if 'analisis_mediadores' in dict_argumentos.keys():
    analisis_mediadores = dict_argumentos['analisis_mediadores']
  else:
    analisis_mediadores = False
    
  if 'row_counting_deviation' in test_executions:
    if 'row_counting_deviation' in dict_thresholds:
      print(row_counting_deviation(df, previous_df, dict_thresholds['row_counting_deviation']))
    else:   
      print(row_counting_deviation(df, previous_df))
  
  if 'column_counting_deviation' in test_executions:
    if 'column_counting_deviation' in dict_thresholds:
      print(column_counting_deviation(df, previous_df, dict_thresholds['column_counting_deviation']))
    else:
      print(column_counting_deviation(df, previous_df))
  
  if 'column_names_checking' in test_executions:
    print(column_names_checking(df, previous_df))
    
  if 'duplicated_key' in test_executions:
    print(duplicated_key(df, dict_args_columns_lists['duplicated_key']))
  
  if 'nulls_check' in test_executions and 'nulls_check' in dict_args_columns_lists:
    print(nulls_check(df, dict_args_columns_lists['nulls_check']))
  elif 'nulls_check' in test_executions:
    print(nulls_check(df))
    
  if 'sum_column_values_deviation' in test_executions:
    if 'sum_column_values_deviation' in dict_thresholds:
      print(sum_column_values_deviation(df, previous_df, dict_args_columns_lists['sum_column_values_deviation'], dict_thresholds['sum_column_values_deviation'], nivel_mediador = analisis_mediadores))
    else:
      print(sum_column_values_deviation(df, previous_df, dict_args_columns_lists['sum_column_values_deviation']), nivel_mediador = analisis_mediadores)
  
  if 'sum_column_aggregated_values_deviation' in test_executions:
    if 'sum_column_aggregated_values_deviation' in dict_thresholds:
      print(sum_column_aggregated_values_deviation(df, previous_df, dict_args_columns_lists['sum_column_aggregated_values_deviation'], dict_args_aggs['sum_column_aggregated_values_deviation'], dict_thresholds['sum_column_aggregated_values_deviation']))
    else:
      print(sum_column_aggregated_values_deviation(df, previous_df, dict_args_columns_lists['sum_column_aggregated_values_deviation'], dict_args_aggs['sum_column_aggregated_values_deviation']))
  
  if 'categories_counting_deviation' in test_executions:
    if 'categories_counting_deviation' in dict_thresholds:
      print(categories_counting_deviation(df, previous_df, dict_args_columns_lists['categories_counting_deviation'], dict_thresholds['categories_counting_deviation']))
    else:
      print(categories_counting_deviation(df, previous_df, dict_args_columns_lists['categories_counting_deviation']))
    
  if 'range_control' in test_executions:
    print(range_control(df, dict_args_columns_lists['range_control'], dict_ranges['range_control']))
    
  if 'variables_type' in test_executions:
    print(variables_type(df, dict_args_columns_lists['variables_type'], dict_types['variables_type']))
  
  if 'total_columns_sum_check' in test_executions:
    print(total_columns_sum_check(df, dict_total_sum))
       
  if 'variable_distribution' in test_executions:
    if 'variable_distribution' in dict_thresholds:
      print(variable_distribution(df, previous_df, dict_args_columns_lists['variable_distribution'], dict_thresholds['variable_distribution']))
    else:
      print(variable_distribution(df, previous_df, dict_args_columns_lists['variable_distribution']))
      
  if 'criteria_pointed_check' in test_executions:
    print(criteria_pointed_check(df, dict_specifications['criteria_pointed_check']))
  
  if 'criteria_pointed_check_planexcelencia' in test_executions:
    print(criteria_pointed_check_planexcelencia(df, dict_specifications['criteria_pointed_check_planexcelencia']))
    

#---------------------------------------------------------------------------------
    
def saving_after_testing(df, environment, output_table_name, unique_key='cod_mediador', carga = 'overwrite'):
  
  """
  Funcion que define la ruta de guardado de un proceso dependiendo del tipo de testeo que se este ejecutando
  
  Parameters:
  ------------
  df : Spark DataFrame
    Dataframe resultado del proceso
  environment : String
    Es el entorno en el que queremos testear el proceso. Puede ser desarrollo para desarrollar, tester para comprobar el codigo antes de subir a produccion, produccion para tests generales de procesos productivos automaticos que guardan en staging. Generalmente vendra dado por los widgets del job
  output_table_name : String
    Ruta en la que se guardara la salida del proceso. Generalmente vendra dado por los widgets del job 
   
  Returns:
  --------
  Un dataframe que se guardara en la ruta dada, devolvera mensaje SUCCESS. Si el entorno se introduce erroneamente devolvera un mensaje de error
  """
  
  # testing_blob_path = '/mnt/datavault/testing/'
  # testing_blob_path_tester = '/mnt/datavault/test_registros_cambios/'
  
  testing_path = 'testing/desarrollo/'
  testing_path_tester = 'testing/tester/'
  contianer_dev_test = "usuarios"
  table_format_dev_test = "parquet"
  
  if environment == 'desarrollo':
    gen_utils.write_staging_table(df, output_table_name, carga, entorno="dev")
    
    return output_table_name
    
  elif environment == 'tester':
    
    if output_table_name == "test_registros_cambios":
      raise ValueError("[ERROR]: La tabla no puede llamarse 'test_registros_cambios'.")
  
    # df.write.mode('overwrite').parquet(testing_blob_path_tester + output_table_name)
    # return testing_blob_path_tester + output_table_name
    
    gen_utils.write_dlake_table(df=df, 
                            table_path=os.path.join(testing_path_tester, output_table_name), 
                            container=contianer_dev_test, 
                            mode=carga, 
                            environment="desarrollo", 
                            table_format=table_format_dev_test)
    
    return output_table_name
    
  
  elif environment == 'produccion':
    #df.write.mode('overwrite').parquet(testing_blob_path + output_table_name)
    #df = spark.read.parquet(testing_blob_path + output_table_name)
    
    #dict_args_columns_lists = {'duplicated_key' : unique_key}
    #tests = ['duplicated_key', 'nulls_check']
    
#     run_all_tests(df, {'tests': tests,
#                        'dict_args_columns_lists': dict_args_columns_lists})  
    gen_utils.write_staging_table(df, output_table_name, carga)
    
    return "SUCCESS"

  else:
    raise SystemExit('El entorno debe ser desarrollo, tester o produccion')
    
#---------------------------------------------------------------------------------

def variables_type(df, columns_list, types_list):
  
  """
  Test que chequea si el formato de una lista de variables coincide con el que nosotros especifiquemos
  
  Parameters:
  ------------
  df : Spark DataFrame
    DataFrame actual
  columns_list : String / list of strings
    Lista con las columnas de las que queremos comprobar el formato
  types_list : type / list of types
    Lista con los formatos de spark SQL en formato types de las variables que queremos chequear. Ej: T.StringType, T.FloatType, etc
  
  Returns:
  --------
  Mensaje de error y la ejecucion se para si hay formatos erroneos, nos indicara el error. Mensaje de OK si pasa el test
  """
  
  if type(columns_list) is str:
    columns_list = [columns_list]
  if type(types_list) is type:
    types_list = [types_list]
  
  for i in range(len(columns_list)):
    for j in range(len(df.schema.names)):
      if df.schema.names[j] == columns_list[i]:
        variable_idx = j
        variable_real_type = df.schema.fields[variable_idx].dataType
    
        if types_list[i]() != variable_real_type:
          raise SystemExit('La variable {0} no tiene tiene formato {1}. Su formato es {2}'.format(columns_list[i], types_list[i], variable_real_type))
    
  return 'Todos los formatos estan OK'

In [0]:
def total_columns_sum_check(df, dict_sum_of_columns): 
  """
  Test que comprueba si la columna que deberia equivaler a la suma de diferentes columnas esta bien calculado. 
  
  Parameters:
  ------------
  df : Spark DataFrame
    DataFrame actual
  dict_sum_of_columns : dict of lists
    Diccionario con claves el nombre de la columna que contiene el total a comprobar y valores listas con las columnas que, sumadas, equivalen a ese total
  
  Returns:
  --------
  Mensaje de error y la ejecucion se para si hay sumas mal calculadas, nos indicara en que columna. Mensaje de OK si pasa el test
  """
  
  for key, value in dict_sum_of_columns.items():
    
    df = df.withColumn('total_sum', sum([df[c] for c in value]))
    totals = df.select('total_sum', key)
    totals = totals.withColumn('difference', F.udf(lambda x, y: 0 if y == 0 else abs((float(x) - float(y)) / float(y)), T.FloatType())(key, 'total_sum'))

    deviated_rows = totals.where(F.col('difference') > 0)

    if deviated_rows.count() > 0:
      deviated_rows.show()
      raise SystemExit('Hay una desviacion en la suma de la lista de columnas con respecto a la columna total en las filas mostradas')
    else:
      print('La suma de valores de columnas con respecto a la columna total {} es correcta'.format(key))

In [0]:
def variable_distribution(df, previous_df, columns_list, deviation_threshold = 0.1, distribution_comparation = False):
  
  """
  Test que pinta los p0 p25 p40 p50 p60 p75 p100 y la media de distribucion. Si dejamos la distribution_comparation se pintara la distribucion del actual df. Si lo pasamos a True lo comparara con la distribucion de previous_df y usara el threshold para limitar las desviaciones
  
  Parameters:
  ------------
  df : Spark DataFrame
    DataFrame actual
  previous_df : Spark DataFrame
    DataFrame previo
  columns_list : String / list of strings
    Lista con las columnas de las que queremos comprobar la distribucion
  deviation_threshold : Float
    Desviacion en decimal
  distribution_comparation : boolean
    Define si comparamos con un dataframe previo o no
  
  Returns:
  --------
  Mensaje de error y la ejecucion se para si hay desviaciones mayores al threshold, nos indicara en que columna. Si no se da un datarame previo se pintara la distribucion del actual. Mensaje de OK si pasa el test
  """
    
  if type(columns_list) != list:
    columns_list = [columns_list]
    
  schema = T.StructType([T.StructField('metricas', T.StringType(), True)])
  metricas = spark.createDataFrame(data = [('p0',), ('p25',), ('p40',), ('p50',), ('p60',), ('p75',), ('p100',), ('mean',)], schema = schema).rdd.zipWithIndex().map(lambda row: (row[1], row[0][0]))
    
  current_quartiles = spark.createDataFrame(pd.DataFrame(ml_pipe.spark_numeric_vars_quantiles(df, columns_list, quantiles = [0., 0.25, 0.4, 0.5, 0.6, 0.75, 1.0])))
  
  columns_list_met = current_quartiles.columns + ['metricas']
  
  current_mean = df.select(current_quartiles.columns).describe().where(F.col('summary') == 'mean').drop('summary')
  current_quartiles = current_quartiles.union(current_mean)
  current_quartiles = current_quartiles.rdd.zipWithIndex().map(lambda row: (row[1], tuple([row[0][i] for i in range(len(columns_list_met) - 1)])))
  current_quartiles = current_quartiles.join(metricas).map(lambda row: [row[1][0][i] for i in range(len(columns_list_met) - 1)] + [row[1][1]]).toDF(columns_list_met)

  if distribution_comparation == False:
    print('Distribucion de las variables numericas:')
    current_quartiles.show()

  else:
    previous_quartiles = spark.createDataFrame(pd.DataFrame(ml_pipe.spark_numeric_vars_quantiles(previous_df, columns_list, quantiles = [0., 0.25, 0.4, 0.5, 0.6, 0.75, 1.0])))
    previous_mean = previous_df.select(current_quartiles.columns).describe().where(F.col('summary') == 'mean').drop('summary')
    previous_quartiles = previous_quartiles.union(previous_mean)
    previous_quartiles = previous_quartiles.rdd.zipWithIndex().map(lambda row: (row[1], tuple([row[0][i] for i in range(len(columns_list_met) - 1)])))
    previous_quartiles = previous_quartiles.join(metricas).map(lambda row: [row[1][0][i] for i in range(len(columns_list_met) - 1)] + [row[1][1]]).toDF(columns_list_met)
  
    for col in previous_quartiles.columns:
      if col != 'metricas':
        previous_quartiles = previous_quartiles.withColumnRenamed(col, col + '_prev')
  
    diff = current_quartiles.join(previous_quartiles, on = 'metricas', how = 'full').na.fill(0)

    for col in diff.columns:
      if col != 'metricas' and '_prev' not in col:
        diff = diff.withColumn(col + '_diff', F.udf(lambda x, y: 0 if y == 0 else abs((float(x) - float(y)) / float(y)))(col, col + '_prev'))

    diff = diff.na.fill(0)

    for col in diff.columns:
      if '_diff' in col:
        if diff.where(F.col(col) > deviation_threshold).count() > 0:
          diff.show()
          raise SystemExit('Hay desviaciones en la distribucion de la variable {}'.format(col))
        elif diff.where(F.col(col) > 0).count() > 0:
          print('Diferencias en la distribucion de la variable numerica {}'.format(col))
          diff.where(F.col(col) > 0).show()
          print('Hay desviaciones en la distribucion de la columna {0} inferiores al limite {1:.0f}%. En la tabla se muestran algunos casos.'.format(column, deviation_threshold * 100))
  
    return 'No hay desviaciones en la distribucion de las variables definidas'

In [0]:
def criteria_pointed_check(df, specifications):
  
  """
  Test que chequea si la puntuacion de los agentes esta bien calculada con respecto a las condiciones que le corresponderian a cada uno de esos agentes. Hay que darle las especificaciones como una lista siguiendo la estructura de la funcion de filtrado de la libreria datavault dv.func_filtros() de la siguiente forma:
  
  - condicion -> tipo: , columna, parametros
  - puntos ->  tipo: , columna, parametros
  - calculo_inverso -> Si False (por defecto) indica que se le agregaran puntos al agente por mejorar un indicador. Si True, se le restaran puntos (siniestralidad por ejemplo, a mayor menos puntos para el agente)
  
  filtros_sini2 = {'condicion': [{'tipo' : 'rango', 'columna' : 'SINI_NO_VIDA', 'parametros' : [40.01, 70.]}],
                 'puntos': [{'tipo' : 'rango', 'columna' : 'PTOS_SINI_NO_VIDA', 'parametros' : [6.66, 199.99]}],
                 'calculo_inverso': True}
                 
  filtros_migenerali1 = {'condicion': [{'tipo' : 'rango', 'columna' : 'RATIO_E_CLIENTES', 'parametros' : ['<=', 15.]}],
                       'puntos': [{'tipo' : 'rango', 'columna' : 'PTOS_E_CLIENTES', 'parametros' : ['=', 0.]}]}
  
  La lista specifications debera contener varios elementos como filtros_sini2 arriba: ejemplo: [filtros_migenerali1, filtros_sini2]
  
  Parameters:
  ------------
  df : Spark DataFrame
    DataFrame actual
  specifications : list of dicts
  
  Returns:
  --------
  Mensaje de error y la ejecucion se para si hay columnas mal puntuadas, nos indicara en que filas. Mensaje de OK si pasa el test
  """
  
  for filtro in specifications:
    a = filtro['condicion'][0]['parametros']
    b = filtro['puntos'][0]['parametros']
    c = filtro['condicion'][0]['columna']
    d = filtro['puntos'][0]['columna']
    
    if 'calculo_inverso' in filtro:
      calculo_inverso = filtro['calculo_inverso']
    else:
      calculo_inverso = None

    if type(a[0]) == str:
      condiciones = dv.func_filtros(df, filtro['condicion']).count() 
      conteo = dv.func_filtros(df, filtro['puntos']).count() 
      
      if conteo == condiciones:
        print('La variable {} esta bien puntuada'.format(c))
      else:
        mostrar = dv.func_filtros(df, filtro['condicion']).union(dv.func_filtros(df, filtro['puntos']))
        mostrar.show()
        raise SystemExit('La variable {0} tiene {1} valores que cumplen las condiciones {2} y {3} valores con los puntos {4}'.format(c, condiciones, a, conteo, b))
      
    else:  
      if calculo_inverso == True:
        if b[0] < 0:
          df = df.withColumn('puntuacion_correcta', F.udf(lambda x, y: 1.0 if abs(((float(x) - (min(a))) * max(b)) - float(y)) <= 1 else 0.0, T.FloatType())(c, d))
        else:
          df = df.withColumn('puntuacion_correcta', F.udf(lambda x, y: 1.0 if abs((max(b) - (float(x) - (min(a))) * min(b)) - float(y)) <= 1 else 0.0, T.FloatType())(c, d))
      else:
        df = df.withColumn('puntuacion_correcta', F.udf(lambda x, y: 1.0 if abs(((float(x) - (min(a))) * min(b)) - float(y)) <= 1 else 0.0, T.FloatType())(c, d))
      
      mostrar = df.where((F.col('puntuacion_correcta') == 0) & (F.col(filtro['condicion'][0]['columna']) <= max(a)) & (F.col(filtro['condicion'][0]['columna']) >= min(a)))
      if mostrar.count() == 0:
        print('La variable {} esta bien puntuada'.format(c))
      else:
        mostrar.show()
        raise SystemExit('La variable {0} no esta bien puntuada para los casos mostrados'.format(c))
    
  return 'Columnas bien puntuadas OK'

In [0]:
def criteria_pointed_check_planexcelencia(df, specifications):
  
  """
  Test que chequea si la puntuacion de los agentes esta bien calculada con respecto a las condiciones que le corresponderian a cada uno de esos agentes. Hay que darle las especificaciones como una lista siguiendo la estructura de la funcion de filtrado de la libreria datavault dv.func_filtros() de la siguiente forma. La diferencia con criteria_pointed_check es que en los rangos intermedios de puntuacion, los incrementos se hacen en 10 puntos basicos, en vez de por punto porcentual:
  
  - condicion -> tipo: , columna, parametros
  - puntos ->  tipo: , columna, parametros
  - calculo_inverso -> Si False (por defecto) indica que se le agregaran puntos al agente por mejorar un indicador. Si True, se le restaran puntos (siniestralidad por ejemplo, a mayor menos puntos para el agente)
  
  filtros_sini2 = {'condicion': [{'tipo' : 'rango', 'columna' : 'SINI_NO_VIDA', 'parametros' : [40.01, 70.]}],
                 'puntos': [{'tipo' : 'rango', 'columna' : 'PTOS_SINI_NO_VIDA', 'parametros' : [6.66, 199.99]}],
                 'calculo_inverso': True}
                 
  filtros_migenerali1 = {'condicion': [{'tipo' : 'rango', 'columna' : 'RATIO_E_CLIENTES', 'parametros' : ['<=', 15.]}],
                       'puntos': [{'tipo' : 'rango', 'columna' : 'PTOS_E_CLIENTES', 'parametros' : ['=', 0.]}]}
  
  La lista specifications debera contener varios elementos como filtros_sini2 arriba: ejemplo: [filtros_migenerali1, filtros_sini2]
  
  Parameters:
  ------------
  df : Spark DataFrame
    DataFrame actual
  specifications : list of dicts
  
  Returns:
  --------
  Mensaje de error y la ejecucion se para si hay columnas mal puntuadas, nos indicara en que filas. Mensaje de OK si pasa el test
  """
  
  for filtro in specifications:
    a = filtro['condicion'][0]['parametros']
    b = filtro['puntos'][0]['parametros']
    c = filtro['condicion'][0]['columna']
    d = filtro['puntos'][0]['columna']
    
    if 'calculo_inverso' in filtro:
      calculo_inverso = filtro['calculo_inverso']
    else:
      calculo_inverso = None

    if type(a[0]) == str:
      if b[1] == 0 and c =='OBJ_POL_EXITO_EO':
        condiciones = df.where((F.col(c) < (a[1])) | (F.col('CLIENTES_VAL') / F.col('OBJ_EX_OPERA_PROP') < 1)).count()
        conteo = dv.func_filtros(df, filtro['puntos']).count() 

        if conteo == condiciones:
          print('La variable {} esta bien puntuada'.format(c))
        else:
          mostrar = dv.func_filtros(df, filtro['condicion']).union(dv.func_filtros(df, filtro['puntos']))
          mostrar.show()
          raise SystemExit('La variable {0} tiene {1} valores que cumplen las condiciones {2} y {3} valores con los puntos {4}'.format(c, condiciones , a, conteo, b))
      elif b[1] == 0:
        condiciones = df.where((F.col(c) < (a[1]))).count()
        conteo = dv.func_filtros(df, filtro['puntos']).count() 

        if conteo == condiciones:
          print('La variable {} esta bien puntuada'.format(c))
        else:
          mostrar = dv.func_filtros(df, filtro['condicion']).union(dv.func_filtros(df, filtro['puntos']))
          mostrar.show()
          raise SystemExit('La variable {0} tiene {1} valores que cumplen las condiciones {2} y {3} valores con los puntos {4}'.format(c, condiciones , a, conteo, b))
        
      
      elif c =='OBJ_POL_EXITO_EO':
        condiciones = df.where((F.col(c) >= a[1])).where(F.col('CLIENTES_VAL') / F.col('OBJ_EX_OPERA_PROP') >= 1).count()
        conteo = dv.func_filtros(df, filtro['puntos']).count() 

        if conteo == condiciones:
          print('La variable {} esta bien puntuada'.format(c))
        else:
          mostrar = dv.func_filtros(df, filtro['condicion']).union(dv.func_filtros(df, filtro['puntos']))
          mostrar.show()
          raise SystemExit('La variable {0} tiene {1} valores que cumplen las condiciones {2} y {3} valores con los puntos {4}'.format(c, condiciones, a, conteo, b))
      else:
        condiciones = df.where((F.col(c) >= a[1])).count()
        conteo = dv.func_filtros(df, filtro['puntos']).count() 

        if conteo == condiciones:
          print('La variable {} esta bien puntuada'.format(c))
        else:
          mostrar = dv.func_filtros(df, filtro['condicion']).union(dv.func_filtros(df, filtro['puntos']))
          mostrar.show()
          raise SystemExit('La variable {0} tiene {1} valores que cumplen las condiciones {2} y {3} valores con los puntos {4}'.format(c, condiciones, a, conteo, b))
      
    else:
      df = df.withColumn('puntuacion_correcta', F.udf(lambda x, y: 1.0 if abs(((float(x) - (min(a))) / 0.1 * ((max(b) - min(b)) / (max(a) - min (a))) * 0.1) + min(b) - float(y)) <= 1 else 0.0, T.FloatType())(c, d))\
      .withColumn('puntuacion_estimada', F.udf(lambda x, y: abs(((float(x) - (min(a))) / 0.1 * ((max(b) - min(b)) / (max(a) - min (a))) * 0.1) + min(b)), T.FloatType())(c, d))
     
      
      mostrar = df.where((F.col('puntuacion_correcta') == 0) & (F.col(filtro['condicion'][0]['columna']) <= max(a)) & (F.col(filtro['condicion'][0]['columna']) >= min(a))
                        & (F.col('CLIENTES_VAL') / F.col('OBJ_EX_OPERA_PROP') >= 1))
      
      if mostrar.count() == 0:
        print('La variable {} esta bien puntuada'.format(c))
      else:
        mostrar.show()
        raise SystemExit('La variable {0} no esta bien puntuada para los casos mostrados'.format(c))
    
  return 'Columnas bien puntuadas OK'

In [0]:
def sort_schema(sche):
  """
  Function that sort by field name a given schema
  
  Arguments:
    - sche (StructType): Initial schema.
    
  Returns:
    - sorted schema.
  """
  
  
  sche_dict = {field.name: idx for idx, field in enumerate(sche)}
  ord_schema = T.StructType([sche[sche_dict[name]] for name in sorted(sche.names)])
  
  return ord_schema

In [0]:
def round_numeric_vars(df, exclude_cols = [], scale = 2):
  """
  Function that rounds the columns of double, float or decimal type to a fixed decimal type for
  a given dataframe:
  
  Arguments:
    - df (Spark DataFrame): Initial Dataframe to be processed
    - exclude_cols (list): List that contains the columns to be excluded from the transformation process
    - scale (int): Indicates the scale to be sused during the rounding procedure
  
  Return:
    - Transformed DataFrame
  """
  
  non_decimal_types = [T.StringType(), T.IntegerType(), T.LongType(), T.DateType()]
  schema = [field for field in df.schema if (field.name not in exclude_cols and field.dataType not in non_decimal_types)]
  for field in schema:
    df = df.withColumn(field.name, F.round(field.name, scale).cast(T.DecimalType(18, scale)))
    
  return df

In [0]:
def check_equal_dfs(df1, df2, exclude_conv_cols=[], scale_num_conv=2):
  """
  Check if two Dataframes are equal. The decimal numerical variables are rounded to
  DecimalType(18, <scale>) so can be compared neglecting negligible differences. 
  Variables not included in the following types are converted: 
    StringType(), IntegerType(), LongType(), DateType().
    
  Args:
    - df1 (DataFrame): first DataFrame to compare.
    - df2 (DataFrame): second DataFrame to compare.
    - exclude_conv_cols (List[String]): list of columns to be excluded from the
      conversion process.
    - scale_num_conv (Integer): scale to be used during the conversion process.
    
  Returns:
    - Boolean, indicating if the two Dataframes are equal
  """
  
  df1, df2 = df1.na.fill("0"), df2.na.fill("0")
  df1_count, df2_count = df1.count(), df2.count()
  schema_df1_s, schema_df2_s = sort_schema(df1.schema), sort_schema(df2.schema)
  
  if (df1_count == df2_count) and (schema_df1_s == schema_df2_s):
  
    df1_c = round_numeric_vars(df1, exclude_cols=exclude_conv_cols, scale=scale_num_conv).select(schema_df1_s.names)
    df2_c = round_numeric_vars(df2, exclude_cols=exclude_conv_cols, scale=scale_num_conv).select(schema_df1_s.names)
    check_count = df1_c.intersect(df2_c).count()
    
    return df1_count == check_count
  
  else:
    
    return False

In [0]:
def saving_after_testing_dlake(df, environment, output_table_name, 
                               table_format, partitioned_by=None, 
                               replace_where=None, container=None,
                               carga ='overwrite'):
  
  """
  Funcion que define la ruta de guardado de un proceso dependiendo del tipo de testeo que se este ejecutando
  
  Parameters:
  ------------
  df : Spark DataFrame
    Dataframe resultado del proceso
  environment : String
    Es el entorno en el que queremos testear el proceso. Puede ser desarrollo para desarrollar, tester para comprobar el codigo antes de subir a produccion, produccion para tests generales de procesos productivos automaticos que guardan en staging. Generalmente vendra dado por los widgets del job
  output_table_name : String
    Ruta en la que se guardara la salida del proceso. Generalmente vendra dado por los widgets del job 
   
  Returns:
  --------
  Un dataframe que se guardara en la ruta dada, devolvera mensaje SUCCESS. Si el entorno se introduce erroneamente devolvera un mensaje de error
  """
  
  testing_path = 'testing/desarrollo/'
  testing_path_tester = 'testing/tester/'
  contianer_dev_test = "usuarios"
  table_format_dev_test = "parquet"
  
  if environment in ['desarrollo', 'dev']:
    
    gen_utils.write_dlake_table(df=df, 
                                table_path=os.path.join(testing_path, output_table_name), 
                                container=contianer_dev_test, 
                                mode=carga, 
                                environment="desarrollo", 
                                table_format=table_format_dev_test, 
                                partitioned_by=partitioned_by)
    
    return output_table_name
    
  elif environment == 'tester':
    
    gen_utils.write_dlake_table(df=df, 
                            table_path=os.path.join(testing_path_tester, output_table_name), 
                            container=contianer_dev_test, 
                            mode=carga, 
                            environment="desarrollo", 
                            table_format=table_format_dev_test, 
                            partitioned_by=partitioned_by, 
                            replace_where=replace_where)
    
    return output_table_name
  
  elif environment in ['produccion', 'pro']:
    
    gen_utils.write_dlake_table(df=df, 
                            table_path=output_table_name, 
                            container=container, 
                            mode=carga, 
                            environment=environment, 
                            table_format=table_format, 
                            partitioned_by=partitioned_by, 
                            replace_where=replace_where)
    
    return "SUCCESS"
    
    
  elif environment in ['pre-produccion', 'pre-pro']:
    
    gen_utils.write_dlake_table(df=df, 
                            table_path=output_table_name, 
                            container=container, 
                            mode=carga, 
                            environment='dev', 
                            table_format=table_format, 
                            partitioned_by=partitioned_by, 
                            replace_where=replace_where)
    
    return "SUCCESS"
    

  else:
    raise SystemExit('El entorno debe ser desarrollo, tester o produccion')